<a href="https://colab.research.google.com/github/Kristin33/10605-mini/blob/main/10605_mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import argparse
import re
import time
from datetime import datetime

import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms

# from torch.utils.data import DataLoader, TensorDataset
# from torch.utils.tensorboard import SummaryWriter
# from torch.optim.lr_scheduler import _LRScheduler

from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.optimizers import RMSprop

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import Isomap, MDS, TSNE
# import matplotlib.pyplot as plt


# from conf import settings
# from utils import get_network, get_training_dataloader, get_test_dataloader, WarmUpLR, \
#     most_recent_folder, most_recent_weights, last_epoch, best_acc_weights

# Data


In [ ]:
# cifar100_train = torchvision.datasets.CIFAR100("cifar100", train=True, download=True)
# cifar100_test = torchvision.datasets.CIFAR100("cifar100", train=False, download=True)
# cifar10_train = torchvision.datasets.CIFAR10("cifar10", train=True, download=True)
# cifar10_test = torchvision.datasets.CIFAR10("cifar10", train=False, download=True)

Extracting cifar100/cifar-100-python.tar.gz to cifar100
Files already downloaded and verified


Extracting cifar10/cifar-10-python.tar.gz to cifar10
Files already downloaded and verified


In [3]:
((cifar100_train_data, cifar100_train_targets), 
 (cifar100_test_data, cifar100_test_targets)) = cifar100.load_data()

In [ ]:
train_data = cifar100_train_data # a numpy array
# train_labels = np.expand_dims(np.array(cifar100_train_targets), 1) 
train_labels = np_utils.to_categorical(cifar100_train_targets)
test_data = cifar100_test_data
# test_labels = np.expand_dims(np.array(cifar100_test_targets), 1) 
test_labels = np_utils.to_categorical(test_labels)
print("Training data shape: ", train_data.shape)
print("Training labels shape: ", train_labels.shape)
print("Testing data shape: ", test_data.shape)

In [ ]:
'''
Transform the data such that each feature has zero mean and 1 std.
Args:
  data (ndarray): must have shape (N, k), where k is the number of features.
'''
def transform(data):
  scaler = StandardScaler()
  transformed_data = scaler.fit_transform(data)
  return transformed_data

def rescale(data):
  return data/255;

In [5]:
# flatten the data
train_data_flattened = train_data.reshape((50000, 3072))
test_data_flattened = test_data.reshape((10000, 3072))

# tranformation
transformed_train_data = train_data_flattened/255.0
transformed_test_data = test_data_flattened/255.0

# Dimensionality Reduction

In [6]:
reduced_dimension = 16

## PCA

In [7]:
time_start = time.time()
pca = PCA(n_components=reduced_dimension) 
pca.fit(transformed_train_data)
reduced_train_data_pca = pca.transform(transformed_train_data)
reduced_test_data_pca = pca.transform(transformed_test_data)
print ('PCA done! Time elapsed: {} seconds'.format(time.time()-time_start))
print("Percentage of variance explained: ", sum(pca.explained_variance_ratio_))
# Note: When n_components=50, 85% of variance is explained.

PCA done! Time elapsed: 13.029044151306152 seconds
Percentage of variance explained:  0.7456769795386474


In [8]:
pca50 = PCA(n_components=50) 
pca50.fit(transformed_train_data)
reduced_train_data_pca50 = pca50.transform(transformed_train_data)
reduced_test_data_pca50 = pca50.transform(transformed_test_data)

## ISOMAP

In [ ]:
time_start = time.time()
isomap = Isomap(n_components=reduced_dimension)
isomap.fit(reduced_train_data_pca50)
reduced_train_data_isomap = isomap.transform(reduced_train_data_pca50[:200, :])
reduced_test_data_isomap = isomap.transform(reduced_test_data_pca50[:200, :])
print ('Isomap done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
plot_distribution(reduced_train_data_isomap, train_labels)

## MDS

In [ ]:
time_start = time.time()
mds = MDS(n_components=reduced_dimension)
mds.fit(reduced_train_data_pca50)
reduced_all_data_mds = mds.fit_transform(
    np.vstack((reduced_train_data_pca50[:200, :], reduced_test_data_pca50[:200, :])))
reduced_train_data_mds = reduced_all_data_mds[:200, :]
reduced_test_data_mds = reduced_all_data_mds[200:, :]
print ('MDS done! Time elapsed: {} seconds'.format(time.time()-time_start))

## T-SNE

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=reduced_dimension)
reduced_all_data_tsne = tsne.fit_transform(
    np.vstack((reduced_train_data_pca50[:200, :], reduced_test_data_pca50[:200, :])))
reduced_train_data_tsne = reduced_all_data_tsne[:200, :]
reduced_test_data_tsne = reduced_all_data_tsne[200:, :]
print ('T-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

NameError: ignored

In [ ]:
print(tsne.n_iter_)

# Run Model

In [9]:
batch_size = 128
num_classes = train_labels.shape[-1]
epochs = 140

In [10]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(reduced_dimension,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              17408     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               25700     
Total params: 1,748,836
Trainable params: 1,748,836
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

print(reduced_train_data_pca.shape)
print(train_labels.shape)
print(reduced_test_data_pca.shape)
print(test_labels.shape)

time_start = time.time()
history = model.fit(reduced_train_data_pca, train_labels, batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(reduced_test_data_pca, test_labels))
print ('Training done! Time elapsed: {} seconds'.format(time.time()-time_start))

(50000, 16)
(50000, 1, 100, 2)
(10000, 16)
(10000, 1, 100, 2)
Epoch 1/140


ValueError: ignored